<p align="center"><img width="50%" src="https://aimodelsharecontent.s3.amazonaws.com/aimodshare_banner.jpg" /></p>


---




<p align="center"><h1 align="center">Quick Start: Sports Clips Video Classification Tutorial</h1> 

---

<h3 align="center">(Deploy model to an AI Model Share Model Playground REST API<br> and Web Dashboard in five easy steps...)</h3></p>
<p align="center"><img width="100%" src="https://aimodelsharecontent.s3.amazonaws.com/aimstutorialsteps.gif" /></p>


---



## **Credential Configuration**

In order to deploy an AI Model Share Model Playground, you will need a credentials text file. 

Generating your credentials file requires two sets of information: 
1. Your AI Model Share username and password (create them [HERE](https://www.modelshare.org/login)). 
2. Your AWS (Amazon Web Services) access keys (follow the tutorial [HERE](https://aimodelshare.readthedocs.io/en/latest/create_credentials.html)). 

You only need to generate your credentials file once. After running the configure function below, save the outputted file for all your future Model Playground deployments and competition submissions. 

*Note: Handle your credentials file with the same level of security you handle your passwords. Do not share your file with anyone, send via email, or upload to Github.*


In [ ]:
#install aimodelshare library
! pip install aimodelshare --upgrade

In [ ]:
# Generate credentials file
import aimodelshare as ai 
from aimodelshare.aws import configure_credentials 

configure_credentials()

AI Modelshare Username:··········
AI Modelshare Password:··········
AWS_ACCESS_KEY_ID:··········
AWS_SECRET_ACCESS_KEY:··········
AWS_REGION:··········
Configuration successful. New credentials file saved as 'credentials.txt'


## **Set up Environment**

Use your credentials file to set your credentials for all aimodelshare functions. 

In [2]:
# Set credentials 
import aimodelshare as ai
from aimodelshare.aws import set_credentials

set_credentials(credential_file="credentials.txt", type="deploy_model")

AI Model Share login credentials set successfully.
AWS credentials set successfully.


In [3]:
# Get materials for tutorial
import aimodelshare as ai
keras_model, y_train_labels = ai.import_quickstart_data("sports")


Data downloaded successfully.

Preparing downloaded files for use...

Success! Your Quick Start materials have been downloaded. 
You are now ready to run the tutorial.


## **(1) Preprocessor Function & Setup**

### **Write a Preprocessor Function**


> ###   Preprocessor functions are used to preprocess data into the precise data your model requires to generate predictions.  

*  *Preprocessor functions should always be named "preprocessor".*
*  *You can use any Python library in a preprocessor function, but all libraries should be imported inside your preprocessor function.*  



In [4]:
# Here is a pre-designed preprocessor, but you could also build your own to prepare the data differently

def preprocessor(video_file_path, num_frames=60, gap_frames=3, **kwargs):

    """
      This function preprocesses the data to extract frames out of each video and resize
      them to a fixed size of (128x128) pixels. Moreover, these images are flattened out to
      act as features for each time step.
      
      params:
          video_file_path
              location of video file to be processed

          num_frames
              the number of frames to be extracted from each video. If there aren't
              sufficient frames, then the list is padded with zeros
              
          gap_frames:
              the number of frames after which we extract the next frame. If =1,
              contiguous frames are extracted
              
      returns:
          X
              transformed features corresponding to data passed as input to model
      
    """

    import cv2
    import numpy as np

    vidcap = cv2.VideoCapture(video_file_path)
    
    frames = []

    success, frame = vidcap.read()
    idx = 0

    while success:
        # for each frame, do if we satisfy the gap_frames parameter
        if idx % gap_frames == 0:
            # convert to RGB (default cv2 is BGR)
            # this is important because the vgg model is trained on RGB images
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame, (128, 128))

            # extract features for the resized image
            frame = frame / 255.0 # Min-max transform.

            # flatten the features and append to a list of features for this video
            frame = frame.reshape(-1)

            frames.append(frame)

            if len(frames) >= num_frames:
                break

        idx += 1
        success, frame = vidcap.read()

    # if number of timesteps or frames < num_frames, pad with zeroes for consistent shape
    while len(frames) < num_frames:
        frames.append(np.zeros(*frames[-1].shape))

    X = np.array(frames)
    X = np.expand_dims(X, axis=0)
    X = np.array(X, dtype=np.float32)

    return X

## **(2) Train Model Using tf.keras (or Your Preferred ML Library)**

### **Keras Model with One LSTM Layer**

In [5]:
#Here is a pre-trained keras_model, but you could also train your own model after preprocessing data with your preprocessor function.
keras_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50)                9840600   
                                                                 
 dense (Dense)               (None, 20)                1020      
                                                                 
 dense_1 (Dense)             (None, 3)                 63        
                                                                 
Total params: 9,841,683
Trainable params: 9,841,683
Non-trainable params: 0
_________________________________________________________________


## **(3) Save Preprocessor**
### Saves preprocessor function to "preprocessor.zip" file

In [6]:
import aimodelshare as ai
ai.export_preprocessor(preprocessor,"") #Saving to your current working directory

Your preprocessor is now saved to 'preprocessor.zip'


In [7]:
#  Now let's import and test the preprocessor function to see if it is working...

import aimodelshare as ai
prep=ai.import_preprocessor("preprocessor.zip")

prep('sports_quick_start_materials/example_data/HorseRace_ex1.avi').shape

(1, 60, 49152)

## **(4) Save Keras Model to ONNX File Format**
...Takes a minute b/c model has nearly 10 million parameters!

In [8]:
# Save tf.keras model (or any tensorflow model) to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(keras_model, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

## **(5) Create your Model Playground and Deploy REST API/ Live Web-Application**

In [9]:
#Set up arguments for Model Playground deployment
import pandas as pd 

model_filepath="model.onnx"
preprocessor_filepath="preprocessor.zip"
exampledata_filepath = "sports_quick_start_materials/example_data" 

In [10]:
from aimodelshare import ModelPlayground

#Instantiate ModelPlayground() Class

myplayground=ModelPlayground(model_type="video", classification=True, private=False)

# Create Model Playground (generates live rest api and web-app for your model/preprocessor)

myplayground.deploy(model_filepath, preprocessor_filepath, y_train_labels, exampledata_filepath)

We need some information about your model before we can build your REST API and interactive Model Playground.
   
Model Name (for AI Model Share Website):Sports Clips Classifier
Model Description (Explain what your model does and 
 why end-users would find your model useful):Video clips of people doing pull-ups, kayaking, and horseback riding. Use this dataset to watch video clips and determine which of three activities are taking place. 
Model Key Words (Search categories that describe your model, separated with commas):sports clips, classification, video
   
Creating your prediction API. (This process may take several minutes.)

[=====================================] Progress: 100% - Complete!                                            

Success! Your Model Playground was created in 71 seconds. 
 Playground Url: "https://bd5gfx9wj3.execute-api.us-east-1.amazonaws.com/prod/m"

You can now use your Model Playground.

Follow this link to explore your Model Playground's functionality
Yo

## **Use your new Model Playground!**

Follow the link in the output above to:
- Generate predictions with your interactive web dashboard
- Access example code in Python, R, and Curl

Or, follow the rest of the tutorial to create a competition for your Model Playground and: 
- Access verified model performance metrics 
- Upload multiple models to a leaderboard 
- Easily compare model performance & structure 

## **Part 2: Create a Competition**

-------

After deploying your Model Playground, you can now create a competition. 

Creating a competition allows you to:
1. Verify the model performance metrics on aimodelshare.org
2. Submit models to a leaderboard
3. Grant access to other users to submit models to the leaderboard
4. Easily compare model performance and structure 

In [11]:
#Download competition data and y_test labels to evaluate model prediction submissions
import aimodelshare as ai
keras_model_2, y_test, y_test_labels = ai.import_quickstart_data("sports", "competition")


Data downloaded successfully.

Preparing downloaded files for use...

Success! Your Quick Start materials have been downloaded. 
You are now ready to run the tutorial.


In [12]:
# save y_train labels to competition data folder 
import shutil 
shutil.copy("sports_quick_start_materials/y_train.csv", "sports_clips_competition_data/y_train.csv")

'sports_clips_competition_data/y_train.csv'

In [ ]:
# Create list of authorized participants for competition
# Note that participants should use the same email address when creating modelshare.org account

emaillist=["emailaddress1@email.com", "emailaddress2@email.com", "emailaddress3@email.com"]

In [13]:
# Create Competition
# Note -- Make competition public (allow any AI Model Share user to submit models) 
# .... by excluding the email_list argument and including the 'public=True' argument 

myplayground.create_competition(data_directory='sports_clips_competition_data', 
                                y_test = y_test_labels, 
                              #  email_list=emaillist)
                               public=True)


--INPUT COMPETITION DETAILS--

Enter competition name:Sports Clips Classification Competition
Enter competition description:Video clips of people doing pull-ups, kayaking, and horseback riding. Use this dataset to watch video clips and determine which of three activities are taking place. 

--INPUT DATA DETAILS--

Note: (optional) Save an optional LICENSE.txt file in your competition data directory to make users aware of any restrictions on data sharing/usage.

Enter data description (i.e.- filenames denoting training and test data, file types, and any subfolders where files are stored):Competition Data is labeled and stored in the data folder.
Enter optional data license descriptive name (e.g.- 'MIT, Apache 2.0, CC0, Other, etc.'):
Uploading your data. Please wait for a confirmation message.

 Success! Model competition created. 

You may now update your prediction API runtime model and verify evaluation metrics with the update_runtime_model() function.

To upload new models and/or p

In [14]:
#Instantiate Competition
#--Note: If you start a new session, the first argument should be the Model Playground url in quotes. 
#--e.g.- mycompetition= ai.Competition("https://2121212.execute-api.us-east-1.amazonaws.com/prod/m)
#See Model Playground "Compete" tab for example model submission code.

mycompetition= ai.Competition(myplayground.playground_url)

In [ ]:
# Add, remove, or completely update authorized participants for competition later
emaillist=["emailaddress4@gmail.com"]

mycompetition.update_access_list(email_list=emaillist,update_type="Add")

Submit Models

In [15]:
#Authorized users can submit new models after setting credentials using modelshare.org username/password

apiurl=myplayground.playground_url # example url from deployed playground: apiurl= "https://123456.execute-api.us-east-1.amazonaws.com/prod/m

from aimodelshare.aws import set_credentials
set_credentials(apiurl=apiurl)

AI Modelshare Username:··········
AI Modelshare Password:··········
AI Model Share login credentials set successfully.


In [16]:
## Prepare X_test data for predictions: 
# Unzip video clips
import zipfile
with zipfile.ZipFile('sports_clips_competition_data/clips_test.zip', 'r') as zip_ref:
    zip_ref.extractall('X_test_clips')

# Preprocess clips
import numpy as np
file_names = [('X_test_clips/clips_test/' + str(i) + '.avi') for i in range(1, 46)]
preprocessed_video_data = [preprocessor(x) for x in file_names]

#Create single X_test array from preprocessed videos
X_test = np.vstack(preprocessed_video_data) 

In [17]:
#Submit Model 1: 

#-- Generate predicted y values (Model 1)
#Note: Keras predict returns the predicted column index location for classification models
prediction_column_index = keras_model.predict(X_test).argmax(axis=1)

# extract correct prediction labels 
predicted_labels = [y_train_labels.columns[i] for i in prediction_column_index]

# Submit Model 1 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=predicted_labels)

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 1

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:1658


In [18]:
# Have a look at architecture for model two
keras_model_2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 60, 5)             983160    
                                                                 
 lstm_1 (LSTM)               (None, 5)                 220       
                                                                 
 dense (Dense)               (None, 3)                 18        
                                                                 
Total params: 983,398
Trainable params: 983,398
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Save Model 2 to .onnx file

# Save tf.keras model to ONNX file
onnx_cnn2 = model_to_onnx(keras_model_2, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

# Save model to local .onnx file
with open("model_2.onnx", "wb") as f:
    f.write(onnx_cnn2.SerializeToString()) 

In [20]:
# Submit Model 2

#-- Generate predicted y values (Model 2)
#Note: Keras predict returns the predicted column index location for classification models
prediction_column_index = keras_model_2.predict(X_test).argmax(axis=1)

# extract correct prediction labels 
predicted_labels = [y_train_labels.columns[i] for i in prediction_column_index]

# Submit Model 2 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model_2.onnx",
                                 prediction_submission=predicted_labels,
                                 preprocessor_filepath="preprocessor.zip")

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 2

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:1658


Get Leaderboard

In [21]:
data = mycompetition.get_leaderboard()
mycompetition.stylize_leaderboard(data)

,accuracy,f1_score,precision,recall,ml_framework,transfer_learning,deep_learning,model_type,depth,num_params,lstm_layers,dense_layers,softmax_act,tanh_act,relu_act,loss,optimizer,memory_size,username,version
0,69.57%,55.46%,52.94%,61.90%,keras,False,True,Sequential,3,983398,2,1,1,2,nan,function,Adam,2218656,AIModelShare,2
1,52.17%,48.16%,45.10%,61.90%,keras,False,True,Sequential,3,9841683,1,2,1,1,1.000000,function,Adam,219360,AIModelShare,1


Compare Models

In [22]:
# Compare two or more models
data=mycompetition.compare_models([1,2],verbose=1)
mycompetition.stylize_compare(data)

,Model_1_Layer,Model_1_Shape,Model_1_Params,Model_2_Layer,Model_2_Shape,Model_2_Params
0,LSTM,"[None, 50]",9840600,LSTM,"[None, 60, 5]",983160
1,Dense,"[None, 20]",1020,LSTM,"[None, 5]",220
2,Dense,"[None, 3]",63,Dense,"[None, 3]",18


#### Check structure of y test data 
(This helps users understand how to submit predicted values to leaderboard)

In [23]:
mycompetition.inspect_y_test()

{'class_balance': {'horserace': 19, 'kayaking': 13, 'pullups': 13},
 'class_labels': ['horserace', 'kayaking', 'pullups'],
 'label_dtypes': {"<class 'str'>": 45},
 'y_length': 45,
 'ytest_example': ['kayaking', 'horserace', 'pullups', 'kayaking', 'kayaking']}

## **Part 3: Maintaining your Model Playground**

-------

Update Runtime model

*Use this function to 1) update the prediction API behind your Model Playground with a new model, chosen from the leaderboard and 2) verify the modelperformance metrics in your Model Playground*

In [24]:
myplayground.update_runtime_model(model_version=2)

['accuracy', 'f1_score', 'precision', 'recall']
{'accuracy': 0.6956521739130435, 'f1_score': 0.5546058879392213, 'precision': 0.5294117647058824, 'recall': 0.6190476190476191}
Runtime model & preprocessor for api: https://bd5gfx9wj3.execute-api.us-east-1.amazonaws.com/prod/m updated to model version 2.

Model metrics are now updated and verified for this model playground.


Delete Deployment 

*Use this function to delete the entire Model Playground, including the REST API, web dashboard, competition, and all submitted models*

In [ ]:
myplayground.delete_deployment()

Running this function will permanently delete all resources tied to this deployment, 
 including the eval lambda and all models submitted to the model competition.

To confirm, type 'permanently delete':permanently delete


'API deleted successfully.'